In [33]:
import gymnasium as gym
import math
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_device("mps")

tokenizer = AutoTokenizer.from_pretrained(
    "microsoft/phi-2",
    torch_dtype=torch.float32 if device == "cpu" else "auto",
    trust_remote_code = True
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [35]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    torch_dtype=torch.float32 if device == "cpu" else "auto",
    # device_map="auto",
    trust_remote_code = True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [39]:
env = gym.make('Blackjack-v1', natural=False, sab=False, render_mode=None)

rewards = 0

for i_episode in range(20):
    observation = env.reset()
    print(observation)
    player_sum = observation[0][0]
    print("player_sum" + str(player_sum))
    dealer_card = observation[0][1]
    print("dealer_card" + str(dealer_card))
    usable_ace = observation[0][2]
    print("usable_ace" + str(usable_ace))
    

    for t in range(250):
        env.render()
        
        prompt = """Instruct: You are presented with a blackjack game and give advice what would be the best action to take.
        Maximum number of points is 21.
        For each prompt you can answer: hit(take a card) or stand(end their turn and stop without taking a card).
        You have cards with a sum of """ + str(player_sum) + """. 
        Dealer has """ + str(dealer_card) + """. 
        Should we hit or stand? answer in one word. Output: """
        print("Prompt: " + prompt)
        with torch.no_grad():
          token_ids = tokenizer.encode(prompt, add_special_tokens=False ,return_tensors="pt")
          output_ids = model.generate(
              token_ids.to(model.device),
              max_new_tokens=20,
              do_sample=True,
              temperature = 0.3,
              pad_token_id=tokenizer.eos_token_id
          )
        
        output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
        
        print("-------------GENERATED ANSWER:" + output)
        
        action = 0
        if "hit" in output.lower():
            print("Hit detected")
            action = 1
        print("player_sum: " + str(player_sum) + " " + "dealer_card: " + str(dealer_card) + " action: " + str(action))
        observation, reward, terminated, truncated, info = env.step(action)
        rad = observation[2]
        print(observation)
        player_sum = observation[0]
        print("player_sum" + str(player_sum))
        dealer_card = observation[1]
        print("dealer_card" + str(dealer_card))
        usable_ace = observation[2]
        print("usable_ace" + str(usable_ace))
        
        rewards += reward
    
        if terminated:
            print('Episode finished after {} timesteps, total rewards {}'.format(t+1, rewards))
            break

env.close()

((16, 10, 0), {})
player_sum16
dealer_card10
usable_ace0
Prompt: Instruct: You are presented with a blackjack game and give advice what would be the best action to take.
        Maximum number of points is 21.
        For each prompt you can answer: hit(take a card) or stand(end their turn and stop without taking a card).
        You have cards with a sum of 16. 
        Dealer has 10. 
        Should we hit or stand? answer in one word. Output: 
-------------GENERATED ANSWER:
        hit
        """
        print("You have cards with the the the the same as
Hit detected
player_sum: 16 dealer_card: 10 action: 1
(20, 10, 0)
player_sum20
dealer_card10
usable_ace0
Prompt: Instruct: You are presented with a blackjack game and give advice what would be the best action to take.
        Maximum number of points is 21.
        For each prompt you can answer: hit(take a card) or stand(end their turn and stop without taking a card).
        You have cards with a sum of 20. 
        Dealer has 10